# Simple cancer binary classification

## Imports

In [1]:
from sklearn.datasets import load_breast_cancer

from neuralnetlib.preprocessing import StandardScaler
from neuralnetlib.activations import Sigmoid, ReLU
from neuralnetlib.layers import Input, Activation, Dense, BatchNormalization
from neuralnetlib.callbacks import EarlyStopping
from neuralnetlib.losses import BinaryCrossentropy
from neuralnetlib.model import Model
from neuralnetlib.optimizers import Adam
from neuralnetlib.metrics import accuracy_score, f1_score, recall_score, precision_score
from neuralnetlib.utils import train_test_split

## 1. Loading a dataset (in this case, Breast Cancer dataset)

In [2]:
data = load_breast_cancer()
X, y = data.data, data.target

## 2. Preprocessing

In [3]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## 3. Model definition

In [4]:
input_neurons = x_train.shape[1:][0]  # Cancer dataset has 30 features
num_hidden_layers = 5  # Number of hidden layers
hidden_neurons = 100  # Number of neurons in each hidden layer
output_neurons = 1  # Binary classification-regression

model = Model()
model.add(Input(input_neurons))
model.add(Dense(hidden_neurons, weights_init='he', random_state=42))
model.add(Activation(ReLU()))

for _ in range(num_hidden_layers - 1):
    model.add(Dense(hidden_neurons, weights_init='he', random_state=42))
    model.add(BatchNormalization())
    model.add(Activation(ReLU()))

model.add(Dense(output_neurons, random_state=42))
model.add(Activation(Sigmoid()))

## 4. Model compilation

In [5]:
model.compile(loss_function=BinaryCrossentropy(), optimizer=Adam(learning_rate=0.0001))

model.summary()

Model
-------------------------------------------------
Layer 1: Input(input_shape=(30,))
Layer 2: Dense(units=100)
Layer 3: Activation(ReLU)
Layer 4: Dense(units=100)
Layer 5: BatchNormalization(momentum=0.99, epsilon=1e-08)
Layer 6: Activation(ReLU)
Layer 7: Dense(units=100)
Layer 8: BatchNormalization(momentum=0.99, epsilon=1e-08)
Layer 9: Activation(ReLU)
Layer 10: Dense(units=100)
Layer 11: BatchNormalization(momentum=0.99, epsilon=1e-08)
Layer 12: Activation(ReLU)
Layer 13: Dense(units=100)
Layer 14: BatchNormalization(momentum=0.99, epsilon=1e-08)
Layer 15: Activation(ReLU)
Layer 16: Dense(units=1)
Layer 17: Activation(Sigmoid)
-------------------------------------------------
Loss function: BinaryCrossentropy
Optimizer: Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
-------------------------------------------------


## 5. Model training

In [10]:
early_stopping = EarlyStopping(patience=5, min_delta=0.001, restore_best_weights=True)

history = model.fit(x_train, y_train, epochs=40, batch_size=48, random_state=42, plot_decision_boundary=True,
          callbacks=[early_stopping])  # Here, the early stopping will stop the training if the loss does not decrease

# You could specify a different metric because loss is the default one

# In this context, accuracy_score would have stopped at epoch 10

[==============================] 100% Epoch 1/40 - loss: 0.4520 -  - 0.04s
[==============================] 100% Epoch 2/40 - loss: 0.4250 -  - 0.08s
[==============================] 100% Epoch 3/40 - loss: 0.4006 -  - 0.07s
[==============================] 100% Epoch 4/40 - loss: 0.3795 -  - 0.07s
[==============================] 100% Epoch 5/40 - loss: 0.3621 -  - 0.07s
[==============================] 100% Epoch 6/40 - loss: 0.3485 -  - 0.06s
[==============================] 100% Epoch 7/40 - loss: 0.3385 -  - 0.06s
[==============================] 100% Epoch 8/40 - loss: 0.3314 -  - 0.06s
[==============================] 100% Epoch 9/40 - loss: 0.3268 -  - 0.06s
[==============================] 100% Epoch 10/40 - loss: 0.3241 -  - 0.07s
[==============================] 100% Epoch 11/40 - loss: 0.3226 -  - 0.07s
[==============================] 100% Epoch 12/40 - loss: 0.3219 -  - 0.06s
[==============================] 100% Epoch 13/40 - loss: 0.3218 -  - 0.06s
Early stopping after 

## 6. Model evaluation

In [7]:
loss = model.evaluate(x_test, y_test)
print(f'Test loss: {loss}')

Test loss: 0.7276956736602257


## 7. Model prediction

In [8]:
y_pred = model.predict(x_test)

## 8. Printing some metrics

In [9]:
accuracy = accuracy_score(y_pred, y_test)
precision = precision_score(y_pred, y_test)
recall = recall_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.9210526315789473
Precision: 0.5
Recall: 0.5
F1 Score: 0.5
